In [1]:
import vertexai
from vertexai import agent_engines
import importlib
import os

from dotenv import load_dotenv, dotenv_values, set_key

In [2]:
load_dotenv()

PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
STAGING_BUCKET = os.environ["STAGING_BUCKET"]
AGENT_DISPLAY_NAME = os.environ["AGENT_DISPLAY_NAME"]
AGENT_FOLDER = os.environ["AGENT_FOLDER"]
PROJECT_NUMBER = os.environ["GOOGLE_CLOUD_PROJECT_NUMBER"]

root_agent = importlib.import_module(f"agent")
root_agent = root_agent.root_agent

app = agent_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

2025-09-29 10:36:55,261 - agent - INFO - MODEL: `gemini-2.5-flash`
2025-09-29 10:36:55,261 - agent - INFO - AGENT_APP_NAME: `natura6`
2025-09-29 10:36:55,262 - agent - INFO - PROJECT_ID: `ml-demos-cyntia`
2025-09-29 10:36:55,262 - agent - INFO - LOCATION: `global`
2025-09-29 10:36:55,263 - agent - INFO - DATA_STORE_ID: `gdrive_1744808233110`
2025-09-29 10:36:55,263 - agent - INFO - AUTH_NAME: `natura_app_auth6`
2025-09-29 10:36:55,264 - agent - INFO - AGENTSPACE_APP_ID: `apagar_1751381593128`


In [3]:
print(f"PROJECT_ID: {PROJECT_ID}")
print(f"LOCATION: {LOCATION}")
print(f"STAGING_BUCKET: {STAGING_BUCKET}")

PROJECT_ID: ml-demos-cyntia
LOCATION: us-central1
STAGING_BUCKET: adk-demo-cyntia


In [4]:
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket="gs://"+STAGING_BUCKET,
)

## Roda local

Vamos primeiro verificar se está funcionando e carregamos a versão correta

In [5]:
query = "Qual a missao da TBG? Caso nao encontre, responda o motivo"
user_id="u_123"

In [6]:
events = []
async for event in app.async_stream_query(
    user_id=user_id,
    message=query,
):
    events.append(event)

# The full event stream shows the agent's thought process
#print("--- Full Event Stream ---")
#for event in events:
#    print(event)

# For quick tests, you can extract just the final text response
final_text_responses = [
    e for e in events
    if e.get("content", {}).get("parts", [{}])[0].get("text")
    and not e.get("content", {}).get("parts", [{}])[0].get("function_call")
]
if final_text_responses:
    print("\n--- Final Response ---")
    print(final_text_responses[0]["content"]["parts"][0]["text"])

2025-09-29 10:36:59,311 - google_adk.google.adk.models.google_llm - INFO - Sending out request, model: gemini-2.5-flash, backend: GoogleLLMVariant.VERTEX_AI, stream: False
2025-09-29 10:36:59,312 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-09-29 10:37:01,481 - google_adk.google.adk.models.google_llm - INFO - Response received from the model.
2025-09-29 10:37:01,483 - google_genai.types - WARNING - Warning: there are non-text parts in the response: ['thought_signature', 'function_call'], returning concatenated text result from text parts. Check the full candidates.content.parts accessor to get the full model response.
2025-09-29 10:37:11,552 - agent - INFO - "Answer":  A missão da TBG (Transportadora Brasileira Gasoduto Bolívia-Brasil S.A.) é "Prover as melhores soluções logísticas conectando as ofertas de gás natural aos mercados, agregando valor em benefício do progresso sustentável da indústria , da geração termelétrica e do aquecimento dos lares.", 


--- Final Response ---
A missão da TBG (Transportadora Brasileira Gasoduto Bolívia-Brasil S.A.) é "Prover as melhores soluções logísticas conectando as ofertas de gás natural aos mercados, agregando valor em benefício do progresso sustentável da indústria, da geração termelétrica e do aquecimento dos lares."


## Deploy

In [14]:
agent_env_vars = dict(dotenv_values(".env"))
keys_to_copy = ["MODEL", "AGENT_APP_NAME", "DATASTORE_LOCATION", "DATASTORE_ID", "AGENT_AUTH_OBJECT_ID", "AGENTSPACE_APP_ID_SEARCH"]
agent_env_vars = dict((k, agent_env_vars[k]) for k in keys_to_copy if k in agent_env_vars)

In [15]:
os.path.join("./", AGENT_FOLDER, 'requirements.txt')

'./RAG_app/requirements.txt'

In [16]:
AGENT_DISPLAY_NAME

'natura6'

In [17]:
agent_env_vars

{'MODEL': 'gemini-2.5-flash'}

In [ ]:
remote_app = agent_engines.create(
    app,
    requirements="RAG_app/requirements.txt",
    extra_packages=["RAG_app/agent.py"],
    display_name=AGENT_DISPLAY_NAME,
    env_vars=agent_env_vars
)

In [18]:
resource_name = remote_app.resource_name
print(f"Resource name: {resource_name}")

Resource name: projects/860802528166/locations/us-central1/reasoningEngines/7597172349141516288


In [19]:
set_key(dotenv_path="../.env", key_to_set="REASONING_ENGINE_ID", value_to_set=resource_name.split('/')[-1])

(True, 'REASONING_ENGINE_ID', '7597172349141516288')

## Teste Agent Engine

Obs: Não vai encontrar o documento se precisar de Autenticação, como gDrive. Vamos apenas testar o endpoint

In [7]:
AGENT_DISPLAY_NAME = os.environ["GOOGLE_CLOUD_PROJECT_NUMBER"]

In [8]:
if 'resource_name' not in locals():
    REASONING_ENGINE_ID = os.environ["REASONING_ENGINE_ID"]
    PROJECT_NUMBER = os.environ["GOOGLE_CLOUD_PROJECT_NUMBER"]
    resource_name = f"projects/{PROJECT_NUMBER}/locations/us-central1/reasoningEngines/{REASONING_ENGINE_ID}"

print(f"Resource name: {resource_name}")
remote_agent = agent_engines.get(resource_name)

Resource name: projects/860802528166/locations/us-central1/reasoningEngines/8848891570573803520


In [9]:
query = "Qual a misao da TBG? Qual a URL retornada?"
user_id="u_123"

In [10]:
events = []
async for event in remote_agent.async_stream_query(
    user_id=user_id,
    message=query,
):
    events.append(event)


# For quick tests, you can extract just the final text response
final_text_responses = [
    e for e in events
    if e.get("content", {}).get("parts", [{}])[0].get("text")
    and not e.get("content", {}).get("parts", [{}])[0].get("function_call")
]
if final_text_responses:
    print("\n--- Final Response ---")
    print(final_text_responses[0]["content"]["parts"][0]["text"])


--- Final Response ---
Não foi possível encontrar a missão da TBG utilizando as ferramentas disponíveis. A URL retornada pela busca é: `https://global-discoveryengine.googleapis.com/v1alpha/projects/860802528166/locations/global/collections/default_collection/engines/apagar_1751381593128/assistants/default_assistant:streamAssist`. Parece ser um endpoint de API interno e não uma URL pública da TBG.


In [32]:
events

[]

## Update Agent

In [14]:
agent_env_vars = dotenv_values("../.env")
del agent_env_vars["GOOGLE_CLOUD_LOCATION"]
del agent_env_vars["GOOGLE_CLOUD_PROJECT"]

In [15]:
if 'resource_name' not in locals():
    REASONING_ENGINE_ID = os.environ["REASONING_ENGINE_ID"]
    PROJECT_NUMBER = os.environ["GOOGLE_CLOUD_PROJECT_NUMBER"]
    resource_name = f"projects/{PROJECT_NUMBER}/locations/us-central1/reasoningEngines/{REASONING_ENGINE_ID}"

print(f"Resource name: {resource_name}")
remote_agent = agent_engines.get(resource_name)

Resource name: projects/860802528166/locations/us-central1/reasoningEngines/2336686509396066304


In [9]:
from vertexai import agent_engines

agent_engines.update(
    resource_name=resource_name,    # Required.
    agent_engine=app,     # Optional.
    requirements="requirements.txt",      # Optional.
    display_name="natura",    # Optional.
    extra_packages=["agent.py"],  # Optional.
    env_vars=agent_env_vars
)

Reading requirements from requirements='requirements.txt'


2025-09-26 09:56:24,304 - vertexai.agent_engines - INFO - Reading requirements from requirements='requirements.txt'


Read the following lines: ['google-cloud-discoveryengine', 'google-cloud-storage ', 'requests', 'pydantic', 'pytest', 'PyYAML', 'python-dotenv', 'google-cloud-aiplatform[adk,agent_engines]', 'google-auth-oauthlib', 'google-genai', 'google-adk', 'cloudpickle']


2025-09-26 09:56:24,306 - vertexai.agent_engines - INFO - Read the following lines: ['google-cloud-discoveryengine', 'google-cloud-storage ', 'requests', 'pydantic', 'pytest', 'PyYAML', 'python-dotenv', 'google-cloud-aiplatform[adk,agent_engines]', 'google-auth-oauthlib', 'google-genai', 'google-adk', 'cloudpickle']


Identified the following requirements: {'cloudpickle': '3.1.1', 'pydantic': '2.11.1', 'google-cloud-aiplatform': '1.114.0'}


...

Update Agent Engine backing LRO: projects/860802528166/locations/us-central1/reasoningEngines/7597172349141516288/operations/5850435486597775360


2025-09-26 09:56:29,163 - vertexai.agent_engines - INFO - Update Agent Engine backing LRO: projects/860802528166/locations/us-central1/reasoningEngines/7597172349141516288/operations/5850435486597775360


Agent Engine updated. Resource name: projects/860802528166/locations/us-central1/reasoningEngines/7597172349141516288


2025-09-26 09:59:40,762 - vertexai.agent_engines - INFO - Agent Engine updated. Resource name: projects/860802528166/locations/us-central1/reasoningEngines/7597172349141516288


resource name: projects/860802528166/locations/us-central1/reasoningEngines/7597172349141516288

## Rode os códigos abaixo

python3 03_register_authorization_resource.py

python3 04_register_agentoauth.py